## Exploratory Analysis for Police Summary Reports

### Packages

In [18]:
# pip install transformers
# !pip install nltk

In [19]:
import os
import re
import random
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Local imports
from text_parser import TextParser

### Parameters

In [25]:
PATH = "C:/Users/fdmol/Desktop/MSCAPP/CAPP30255/NLP-Police-Complaints/data/text_files"

#### Functions

I define a class for reading and processing the data, I took some ideas from Matt's analysis to remove headers and other elements that are not relevant to us. This could also help in getting better results for the summarization task.

The function below wraps HuggingFace's tokenizer and model to generate a summary for each complaint. As I mention below, I tweaked the parameters to get better summaries.

In [28]:
def generate_summary(complaint_text, model_name):
    """
    Generates a summary of a complaint given
    the complaint text
    """

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    # Tokenize the text
    inputs = tokenizer(
        complaint_text, return_tensors="pt", max_length=2512, truncation=True
    )

    # Generate summary
    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=1200,
        min_length=40,
        length_penalty=2.0,
        no_repeat_ngram_size=2,
        num_beams=4,
        early_stopping=True,
    )

    # Decode and print the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

#### NLP Task: Summarization

I will use this model:

https://huggingface.co/docs/transformers/main/en/model_doc/t5#transformers.T5ForConditionalGeneration

In the cells below, I use the model to generate a summary for ten random complaints. The summaries are of medium quality, depending on each complaint. 

I did the following to try to improve the quality of the summaries:

- Adjuster the `max_length` parameter to limit the length of the summary
- Adjusted the `min_length` parameter to ensure the summary is at least a certain length
- Adjusted the `num_beams` parameter to increase the number of beams used in beam search
- Adjusted `no_repeat_ngram_size` parameter to avoid repeating n-grams in the summary

I also experimented with the max_length of the tokens used in the tokenizer.

In [29]:
model_name_falcon = "Falconsai/text_summarization"
model_name_bart = "facebook/bart-large-cnn"


In [33]:
text_parser = TextParser(PATH, nlp_task="summarization")

# Get a random list of 10 complaints
complaints = os.listdir(PATH)
complaints = [complaint for complaint in complaints if complaint.endswith(".txt")]
complaints = random.sample(complaints, 3)


for complaint in complaints:
    complaint_text = text_parser.file_to_string(complaint)
    print(f"Model: {model_name_falcon}")
    summary = generate_summary(complaint_text, model_name_falcon)
    print(f"Complaint: {complaint}")
    print("=====================================")
    print(f"Summary: {summary}\n\n")

    # print(f"Model: {model_name_bart}")
    # summary = generate_summary(complaint_text, model_name_bart)
    # print(f"Summary: {summary}\n\n")

Model: Falconsai/text_summarization
Complaint: 2018-1088298.txt
Summary: alleged officer failed to activate his body worn camera during the arrest of in violation of rule s 1 and 2. ms. stated that while in the back of a police squadrol the accused used excessive force against in that he punched him in her face, choked him, and poked his eye as the officer yelling back “stop biting me and stop kicking me.”11 she rela ted that officer threatened to tase him if she didn’t comply, but said 


Model: Falconsai/text_summarization
Complaint: 2018-1090124.txt
Summary: june 28, 2018, at 7:55 a.m., at or near 4650 w. harrison street, chicago, il july 6, 2018 3:42. copa,4 accused field training officer carla jackson. she said she was waiting for the bus when she encountered his cousin, saying he had done nothing wrong. police arrived on scene and surrounded by multiple officers, including fto, said that they had their guns drawn and attempted to fight, shoot, and kill


Model: Falconsai/text_sum

Some next steps could be:

- Improve cleaning process to remove irrelevant headers and footers
- Try other models
- Finetune a model on this dataset (we would need to create a labeled dataset for this, and possibly generate the summaries by hand).